# Data Cleaning
-------------

We are going to clean and merge three datasets:
 <br>raw_rawg.csv----  Key data: metacritic, rating, playtime
 <br>vgsales.csv----  Key data: sales
 <br>steam.csv----   Key data: price

In [1]:
import pandas as pd
import re
import numpy as np
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import IterativeImputer

pd.set_option('display.max_columns', None)

# 1. Merge the first two datasets

In [2]:
df_rawg = pd.read_csv("raw_rawg.csv")     
df_sales = pd.read_csv("vgsales.csv")  

In [3]:
df_rawg.head()

,name,released,platforms,metacritic,rating,ratings_count,playtime,tags,genres
0,Half-Life,1998-11-19,"['PC', 'macOS', 'Linux', 'PlayStation 2', 'Dre...",96.0,4.37,2353,3,"['Singleplayer', 'Для одного игрока', 'Экшен',...","['Shooter', 'Action']"
1,Fallout,1997-09-30,"['PC', 'macOS', 'Linux', 'Classic Macintosh']",NaN,4.20,1199,1,"['Singleplayer', 'Для одного игрока', 'Приключ...",['RPG']
2,Half-Life: Opposing Force,1999-11-19,"['PC', 'macOS', 'Linux']",NaN,3.85,734,3,"['Singleplayer', 'Для одного игрока', 'Экшен',...","['Shooter', 'Action']"
3,Resident Evil 2 (1998),1998-01-21,"['PC', 'PlayStation', 'GameCube', 'Nintendo 64...",90.0,4.32,909,13,"['Singleplayer', 'Atmospheric', 'Great Soundtr...",['Action']
4,Team Fortress Classic,1999-04-01,"['PC', 'macOS', 'Linux']",NaN,2.92,382,1,"['Экшен', 'Приключение', 'Multiplayer', 'Для н...",['Action']


In [4]:
df_rawg.shape

(53607, 9)

In [5]:
df_sales.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [6]:
df_sales.shape

(16598, 11)

In [7]:
df_sales.rename(columns={"Name": "name"}, inplace=True)

**Define a clean_title function to create a clean name for each game. Then, we'll merge the two datasets on the clean_title.**

In [8]:
def clean_title(title):
    if pd.isnull(title):
        return ""
    # Convert to lowercase
    title = title.lower()
    # Remove punctuation or special characters
    title = re.sub(r"[^\w\s]", "", title)
    # Trim whitespace
    title = title.strip()
    return title

df_rawg["clean_name"] = df_rawg["name"].apply(clean_title)
df_sales["clean_name"] = df_sales["name"].apply(clean_title)


In [9]:
df_merged = pd.merge(
    df_rawg, 
    df_sales, 
    on="clean_name",    
    how="inner"         # inner join
)


In [10]:
print("Merged Dataset shape:", df_merged.shape)
# df_merged.to_csv("merged_data.csv", index=False)

Merged Dataset shape: (10227, 21)


In [11]:
df_merged.drop(columns=['platforms', 'Year','genres', 'Rank', 'name_y'], inplace=True)
df_merged.head(10)

,name_x,released,metacritic,rating,ratings_count,playtime,tags,clean_name,Platform,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,Half-Life,1998-11-19,96.0,4.37,2353,3,"['Singleplayer', 'Для одного игрока', 'Экшен',...",halflife,PC,Shooter,Vivendi Games,4.03,0.00,0.09,0.00,4.12
1,Half-Life,1998-11-19,96.0,4.37,2353,3,"['Singleplayer', 'Для одного игрока', 'Экшен',...",halflife,PS2,Shooter,Vivendi Games,0.29,0.23,0.00,0.08,0.60
2,Oddworld: Abe's Oddysee,1997-09-19,85.0,3.53,555,3,"['Singleplayer', 'Для одного игрока', 'Экшен',...",oddworld abes oddysee,PS,Platform,GT Interactive,0.75,0.44,0.00,0.06,1.24
3,Resident Evil,1996-03-22,91.0,4.18,788,4,"['gameplay feature: multiple endings', 'playst...",resident evil,PS,Action,Virgin Interactive,2.05,1.16,1.11,0.73,5.05
4,Resident Evil,1996-03-22,91.0,4.18,788,4,"['gameplay feature: multiple endings', 'playst...",resident evil,SAT,Action,Capcom,0.00,0.00,0.17,0.00,0.17
5,Resident Evil,1996-03-22,91.0,4.18,788,4,"['gameplay feature: multiple endings', 'playst...",resident evil,PS3,Action,Capcom,0.00,0.00,0.12,0.00,0.12
6,X-COM: UFO Defense,1993-12-30,NaN,3.65,275,1,"['Singleplayer', 'Для одного игрока', 'Atmosph...",xcom ufo defense,PS,Strategy,Microprose,0.05,0.04,0.00,0.01,0.09
7,Quake,1996-06-22,94.0,4.25,474,1,"['Singleplayer', 'Multiplayer', 'Steam Cloud',...",quake,N64,Shooter,GT Interactive,0.15,0.04,0.00,0.00,0.19
8,Tomb Raider II,1997-10-31,69.0,3.74,298,1,"['Singleplayer', 'Для одного игрока', 'Экшен',...",tomb raider ii,PS,Action,Eidos Interactive,2.30,2.46,0.20,0.28,5.24
9,Tomb Raider II,1997-10-31,69.0,3.74,298,1,"['Singleplayer', 'Для одного игрока', 'Экшен',...",tomb raider ii,PC,Action,Eidos Interactive,0.91,1.25,0.00,0.13,2.29


**Now, we have a merged_data.csv that is merged from raw_rawg.csv and vgsales.csv. The two datasets are merged on clean_title using inner join. We have 10227 rows of games and 21 columns of features.**

# 2. Merge the third dataset to the first two merged dataset

In [21]:
df_steam = pd.read_csv("steam.csv")
df_steam.head()

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99


In [22]:
df_steam.shape

(27075, 18)

In [23]:
df_steam["clean_name"] = df_steam["name"].apply(clean_title)

In [24]:
df_final = pd.merge(
    df_merged,
    df_steam,
    on="clean_name",  
    how="left"    # left join
)

df_final.shape

(10237, 34)

In [25]:
df_final.head()

,name_x,released,metacritic,rating,ratings_count,playtime,tags,clean_name,Platform,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,Half-Life,1998-11-19,96.0,4.37,2353,3,"['Singleplayer', 'Для одного игрока', 'Экшен',...",halflife,PC,Shooter,Vivendi Games,4.03,0.00,0.09,0.00,4.12,70.0,Half-Life,1998-11-08,1.0,Valve,Valve,windows;mac;linux,0.0,Single-player;Multi-player;Online Multi-Player...,Action,FPS;Classic;Action,0.0,27755.0,1100.0,1300.0,83.0,5000000-10000000,7.19
1,Half-Life,1998-11-19,96.0,4.37,2353,3,"['Singleplayer', 'Для одного игрока', 'Экшен',...",halflife,PS2,Shooter,Vivendi Games,0.29,0.23,0.00,0.08,0.60,70.0,Half-Life,1998-11-08,1.0,Valve,Valve,windows;mac;linux,0.0,Single-player;Multi-player;Online Multi-Player...,Action,FPS;Classic;Action,0.0,27755.0,1100.0,1300.0,83.0,5000000-10000000,7.19
2,Oddworld: Abe's Oddysee,1997-09-19,85.0,3.53,555,3,"['Singleplayer', 'Для одного игрока', 'Экшен',...",oddworld abes oddysee,PS,Platform,GT Interactive,0.75,0.44,0.00,0.06,1.24,15700.0,Oddworld: Abe's Oddysee®,2008-08-28,1.0,Oddworld Inhabitants,Oddworld Inhabitants,windows,0.0,Single-player;Steam Trading Cards,Adventure,Adventure;Platformer;Classic,0.0,2739.0,442.0,387.0,217.0,2000000-5000000,1.99
3,Resident Evil,1996-03-22,91.0,4.18,788,4,"['gameplay feature: multiple endings', 'playst...",resident evil,PS,Action,Virgin Interactive,2.05,1.16,1.11,0.73,5.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Resident Evil,1996-03-22,91.0,4.18,788,4,"['gameplay feature: multiple endings', 'playst...",resident evil,SAT,Action,Capcom,0.00,0.00,0.17,0.00,0.17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
df_final.drop(columns=['release_date', 'name', 'platforms', 'developer', 'publisher', 'appid', 'tags', 'clean_name', 'english', 'genres', 'owners', 'steamspy_tags'], inplace=True)  # if game_name is your unified name

df_final.rename(columns={'name_x': 'name', 'NA_Sales': 'NA_Sales(MM)', 'EU_Sales': 'EU_Sales(MM)', 'JP_Sales': 'JP_Sales(MM)', 'Other_Sales': 'Other_Sales(MM)', 'Global_Sales': 'Global_Sales(MM)'}, inplace=True)

print(df_final.isnull().sum())

print(df_final.describe(include='all'))



name                   0
released              39
metacritic          5836
rating                 0
ratings_count          0
playtime               0
Platform               0
Genre                  0
Publisher             15
NA_Sales(MM)           0
EU_Sales(MM)           0
JP_Sales(MM)           0
Other_Sales(MM)        0
Global_Sales(MM)       0
required_age        8701
categories          8701
achievements        8701
positive_ratings    8701
negative_ratings    8701
average_playtime    8701
median_playtime     8701
price               8701
dtype: int64
                               name    released   metacritic        rating  \
count                         10237       10198  4401.000000  10237.000000   
unique                         6057        2923          NaN           NaN   
top     Need For Speed: Most Wanted  2011-11-15          NaN           NaN   
freq                             12          45          NaN           NaN   
mean                            NaN         NaN

In [27]:
df_final = df_final.dropna(subset=['released', 'Publisher', 'metacritic'])

print(df_final.isnull().sum())

name                   0
released               0
metacritic             0
rating                 0
ratings_count          0
playtime               0
Platform               0
Genre                  0
Publisher              0
NA_Sales(MM)           0
EU_Sales(MM)           0
JP_Sales(MM)           0
Other_Sales(MM)        0
Global_Sales(MM)       0
required_age        3132
categories          3132
achievements        3132
positive_ratings    3132
negative_ratings    3132
average_playtime    3132
median_playtime     3132
price               3132
dtype: int64


In [28]:
df_final.head(10)

,name,released,metacritic,rating,ratings_count,playtime,Platform,Genre,Publisher,NA_Sales(MM),EU_Sales(MM),JP_Sales(MM),Other_Sales(MM),Global_Sales(MM),required_age,categories,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,price
0,Half-Life,1998-11-19,96.0,4.37,2353,3,PC,Shooter,Vivendi Games,4.03,0.00,0.09,0.00,4.12,0.0,Single-player;Multi-player;Online Multi-Player...,0.0,27755.0,1100.0,1300.0,83.0,7.19
1,Half-Life,1998-11-19,96.0,4.37,2353,3,PS2,Shooter,Vivendi Games,0.29,0.23,0.00,0.08,0.60,0.0,Single-player;Multi-player;Online Multi-Player...,0.0,27755.0,1100.0,1300.0,83.0,7.19
2,Oddworld: Abe's Oddysee,1997-09-19,85.0,3.53,555,3,PS,Platform,GT Interactive,0.75,0.44,0.00,0.06,1.24,0.0,Single-player;Steam Trading Cards,0.0,2739.0,442.0,387.0,217.0,1.99
3,Resident Evil,1996-03-22,91.0,4.18,788,4,PS,Action,Virgin Interactive,2.05,1.16,1.11,0.73,5.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Resident Evil,1996-03-22,91.0,4.18,788,4,SAT,Action,Capcom,0.00,0.00,0.17,0.00,0.17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Resident Evil,1996-03-22,91.0,4.18,788,4,PS3,Action,Capcom,0.00,0.00,0.12,0.00,0.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Quake,1996-06-22,94.0,4.25,474,1,N64,Shooter,GT Interactive,0.15,0.04,0.00,0.00,0.19,0.0,Single-player;Multi-player;Co-op;Steam Cloud,0.0,2618.0,136.0,167.0,189.0,3.99
8,Tomb Raider II,1997-10-31,69.0,3.74,298,1,PS,Action,Eidos Interactive,2.30,2.46,0.20,0.28,5.24,12.0,Single-player,0.0,805.0,115.0,3.0,3.0,4.99
9,Tomb Raider II,1997-10-31,69.0,3.74,298,1,PC,Action,Eidos Interactive,0.91,1.25,0.00,0.13,2.29,12.0,Single-player,0.0,805.0,115.0,3.0,3.0,4.99
19,Crash Bandicoot 3: Warped,1998-10-31,91.0,4.34,641,5,PS,Platform,Sony Computer Entertainment,3.68,1.75,1.42,0.28,7.13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
df_final.columns

Index(['name', 'released', 'metacritic', 'rating', 'ratings_count', 'playtime',
       'Platform', 'Genre', 'Publisher', 'NA_Sales(MM)', 'EU_Sales(MM)',
       'JP_Sales(MM)', 'Other_Sales(MM)', 'Global_Sales(MM)', 'required_age',
       'categories', 'achievements', 'positive_ratings', 'negative_ratings',
       'average_playtime', 'median_playtime', 'price'],
      dtype='object')

In [30]:
# Drop useless columns 
df_final.drop(columns=['required_age',
       'categories', 'achievements', 'positive_ratings', 'negative_ratings',
       'average_playtime', 'median_playtime'], inplace=True)


# df_final['has_rating'] = df_final['ratings_count'] > 0

print(df_final.isnull().sum())

name                   0
released               0
metacritic             0
rating                 0
ratings_count          0
playtime               0
Platform               0
Genre                  0
Publisher              0
NA_Sales(MM)           0
EU_Sales(MM)           0
JP_Sales(MM)           0
Other_Sales(MM)        0
Global_Sales(MM)       0
price               3132
dtype: int64


In [31]:
df_final.shape

(4388, 15)

Add Platform Mapping

In [32]:
#define groupings
platform_group_map = {
    'PS': 'PlayStation Consoles', 'PS2': 'PlayStation Consoles', 'PS3': 'PlayStation Consoles', 'PS4': 'PlayStation Consoles',
    'XB': 'Xbox Consoles', 'X360': 'Xbox Consoles', 'XOne': 'Xbox Consoles',
    'NES': 'Nintendo Consoles', 'SNES': 'Nintendo Consoles', 'N64': 'Nintendo Consoles', 'GC': 'Nintendo Consoles',
    'Wii': 'Wii Series', 'WiiU': 'Wii Series',
    'PSP': 'Handhelds', 'PSV': 'Handhelds', 'GB': 'Handhelds', 'GBA': 'Handhelds', 'DS': 'Handhelds', '3DS': 'Handhelds', 'WS': 'Handhelds',
    'PC': 'PC',
    'GEN': 'Other', 'SAT': 'Other', 'DC': 'Other', 'SCD': 'Other', '3DO': 'Other', 'NG': 'Other', '2600': 'Other'
}

#map platforms to groups
df_final['Platform_Group'] = df_final['Platform'].map(platform_group_map)
df_final['Platform_Group'].value_counts()

Platform_Group
PlayStation Consoles    1436
Xbox Consoles            967
Handhelds                847
PC                       575
Wii Series               306
Nintendo Consoles        233
Other                     24
Name: count, dtype: int64

In [33]:
print(df_final.duplicated().sum())

4


In [34]:
df_final.drop_duplicates(inplace=True)
print(df_final.duplicated().sum())

0


In [35]:
df_final.to_csv("final_dataset.csv", index=False, encoding="utf-8")

In [36]:
df_final.name.nunique()

2217

**At the end, we have a final_dataset.csv that is merged from merged_data.csv and steam.csv. The two datasets are merged on clean_title using left join where all the rows are kept from the merged_data.csv dataset even though they are not a common game in both dataset. This is because there are only ~1500 games in common from the two datasets. In order to keep as much data as possible, We had to use left join. We end up having 4388 rows of games and 17 columns of features.**